# Term Frequency Inverse Document Frequency (TF) Vector Representation
__Anish Sachdeva (DTU/2K16/MC/013)__

__Natural Language Processing (IT-425)__

In this noteook we will extract Term Frequency vector representations from a given corpus, where our corpus will be my resume. We will divide the corpus into 6 different parts and each part will be treated as a document. The vector for a given word will be a $1 \times 6$ vector and each column will represent the frequency countof how many times the word occured in that particular document. 

## 1. Importing Required Packages

In [6]:
import pprint
from collections import Counter

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
import pandas

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 2. Importing the Corpus (Resume)

In [4]:
resume_file = open('../assets/resume.txt', 'r')
resume = resume_file.read().lower()
resume_file.close()
print(resume)

anish sachdeva
software developer + clean code enthusiast

phone : 8287428181
email : anish_@outlook.com
home : sandesh vihar, pitampura, new delhi - 110034
date of birth : 7th april 1998
languages : english, hindi, french

work experience
what after college (4 months)
delhi, india
creating content to teach core java and python with data structures and algorithms and giving online classes to students.
giving python classes workshops to students all around india and teaching core data structures and the python api
with emphasis on data structures, algorithms and problem solving. see a sample python batch here:
https://github.com/anishlearnstocode/python-workshop-6

also teaching java to students in batches of 10 days, where the full java api and data types are covered along with many
important algorithms are aso taught. see a sample java batch here: https://github.com/anishlearnstocode/java-wac-batch-32

summer research fellow at university of auckland (2 months)
auckland, new zealand
w

## 3. Tokenizing The Resume
We now create a utility function called `tokenize` that will take in a corpus (resume in this case) and will return us a list of tokens after removing stopwords and punctuations. It will only consider alphabetic words and all numbers have also been ignored.  

In [7]:
# utility function for tokenizing
def tokenize(document: str, stopwords_en=stopwords.words('english'), tokenizer=nltk.RegexpTokenizer(r'\w+')):
    document = document.lower()
    return [token for token in tokenizer.tokenize(document) if token not in stopwords_en and token.isalpha()]

In [8]:
# tokenizing the resume
tokens = tokenize(resume)

# see first 30 tokens
print(tokens[: 30])

['anish', 'sachdeva', 'software', 'developer', 'clean', 'code', 'enthusiast', 'phone', 'email', 'outlook', 'com', 'home', 'sandesh', 'vihar', 'pitampura', 'new', 'delhi', 'date', 'birth', 'april', 'languages', 'english', 'hindi', 'french', 'work', 'experience', 'college', 'months', 'delhi', 'india']


## 4. Dividing the Corpus Into 6 Documents

In [9]:
k = len(tokens) // 6
documents = []
for i in range(5):
    documents.append(tokens[i * k: (i + 1) * k])
documents.append(tokens[4 * k:])

In [10]:
# the 6th document is 
print(documents[5])

['course', 'content', 'quizzes', 'online', 'platform', 'coding', 'ninjas', 'java', 'helped', 'students', 'core', 'data', 'structures', 'algorithms', 'concepts', 'java', 'education', 'delhi', 'technological', 'university', 'bachelors', 'technology', 'mathematics', 'computing', 'cgpa', 'heritage', 'school', 'rohini', 'physics', 'chemistry', 'maths', 'computer', 'science', 'english', 'senior', 'secondary', 'secondary', 'cgpa', 'technical', 'skills', 'java', 'algorithms', 'data', 'structures', 'mean', 'stack', 'web', 'development', 'python', 'machine', 'learning', 'matlab', 'octave', 'mysql', 'postgressql', 'mongodb', 'skills', 'ms', 'office', 'adobe', 'photoshop', 'latex', 'mitex', 'university', 'courses', 'applied', 'mathematics', 'ii', 'iii', 'linear', 'algebra', 'probability', 'statistics', 'stochastic', 'processes', 'discrete', 'maths', 'computer', 'organization', 'architecture', 'data', 'structures', 'algorithm', 'design', 'analysis', 'dbms', 'os', 'computer', 'vision', 'nlp', 'impor

## 5. Calculating Most Common 5 Tokens From Each Document & Storing Frequency Tables for Each Document

In [11]:
most_common = set()
document_frequencies = []
for document in documents:
    frequencies = Counter(document)
    document_frequencies.append(frequencies)
    for word, frequency in frequencies.most_common(5):
        most_common.add(word)

In [12]:
# number of tokens we have selected, as it isn't necessary to obtain 30 unique tokens
print('Number of tokens:', len(most_common))

Number of tokens: 25


In [14]:
# The tokens from the first document are
print('Tokens from first document:', document_frequencies[0].most_common(5))

Tokens from first document: [('python', 5), ('data', 3), ('structures', 3), ('students', 3), ('com', 2)]


In [15]:
# The selected tokens are
pprint.pp(most_common)

{'algorithms',
 'also',
 'applications',
 'auckland',
 'cern',
 'college',
 'com',
 'data',
 'geometry',
 'group',
 'guitar',
 'java',
 'london',
 'mathematics',
 'participated',
 'plectrum',
 'python',
 'requests',
 'research',
 'structures',
 'students',
 'theory',
 'trinity',
 'university',
 'worked'}


## 6. Calculating Number of Documents a Keyword Appears In
The TF-IDF vector for a given word is given by:
$$
tfidf(w, d) = tf(w, d) \times idf(w, d) \\
idf(w, d) = \log{\frac{N_t}{N_w}}
$$

where:

$N_t:$ is the total numeber of documents and

$N_w:$ is the total number of documents containing the keyword $w$.

We now create a dictionary `N_w` (_str_ $\rightarrow$ _int_ ) which will store the number of docuemnts a word $w$ occurrs in.

In [16]:
N_t = 6
N_w = {}
for word in most_common:
    count = 0
    for frequencies in document_frequencies:
        count = count + (word in frequencies)
    N_w[word] = count

In [17]:
# seeing the N_w map for all the selected words
pprint.pp(N_w)

{'requests': 1,
 'geometry': 1,
 'mathematics': 3,
 'university': 3,
 'algorithms': 4,
 'java': 6,
 'college': 2,
 'group': 2,
 'com': 3,
 'theory': 2,
 'python': 3,
 'plectrum': 1,
 'students': 3,
 'london': 1,
 'research': 3,
 'cern': 3,
 'trinity': 1,
 'participated': 1,
 'guitar': 1,
 'data': 4,
 'applications': 1,
 'worked': 3,
 'also': 3,
 'structures': 3,
 'auckland': 2}


We notice above that __java__ is the only word in the given list to appear in all 6 documents.

## 7. Computing the TF-IDF Vectors

In [19]:
vectors = {}
for word in most_common:
    vector = [0] * 6
    for index, frequencies in enumerate(document_frequencies):
        vector[index] = frequencies[word] * np.log(N_t / N_w[word])
    vectors[word] = vector

In [20]:
# Let's see the vector output for a few words
print(vectors['java'])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [21]:
print(vectors['students'])

[2.0794415416798357, 0.0, 0.0, 0.0, 0.6931471805599453, 0.6931471805599453]


In [22]:
# you can also test it out with a word of your choice, try below:
word = 'python'
print(vectors.get(word, [0] * 6))

[3.4657359027997265, 0.0, 0.0, 0.0, 0.6931471805599453, 0.6931471805599453]


## 8. Reprsemting The Vectors in a Tabular Form 

In [23]:
table = pandas.DataFrame(data=vectors)

In [30]:
print(table.iloc[:, 0:7])

   requests  geometry  mathematics  university  algorithms  java   college
0  0.000000  0.000000     0.000000    0.000000    0.810930   0.0  1.098612
1  0.000000  5.375278     2.079442    1.386294    0.405465   0.0  0.000000
2  0.000000  0.000000     0.000000    0.000000    0.000000   0.0  0.000000
3  3.583519  0.000000     0.000000    0.000000    0.000000   0.0  0.000000
4  0.000000  0.000000     1.386294    1.386294    0.810930   0.0  0.000000
5  0.000000  0.000000     2.079442    2.079442    0.810930   0.0  4.394449


In [31]:
print(table.iloc[:, 7:14])

      group       com    theory    python  plectrum  students    london
0  0.000000  1.386294  0.000000  3.465736  0.000000  2.079442  0.000000
1  1.098612  0.693147  3.295837  0.000000  0.000000  0.000000  0.000000
2  2.197225  0.000000  1.098612  0.000000  0.000000  0.000000  0.000000
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
4  0.000000  0.000000  0.000000  0.693147  0.000000  0.693147  0.000000
5  0.000000  2.079442  0.000000  0.693147  7.167038  0.693147  7.167038


In [34]:
print(table.iloc[:, 14:20])

   research      cern   trinity  participated    guitar      data
0  0.000000  0.000000  0.000000      0.000000  0.000000  1.216395
1  0.693147  0.000000  0.000000      0.000000  0.000000  0.405465
2  1.386294  2.079442  0.000000      0.000000  0.000000  0.000000
3  0.000000  1.386294  0.000000      3.583519  0.000000  0.000000
4  0.000000  0.000000  0.000000      0.000000  0.000000  0.810930
5  0.693147  1.386294  7.167038      0.000000  7.167038  1.216395


In [36]:
print(table.iloc[:, 20:])

   applications    worked      also  structures  auckland
0      0.000000  0.000000  0.693147    2.079442  0.000000
1      0.000000  1.386294  0.000000    0.000000  3.295837
2      5.375278  2.079442  0.693147    0.000000  0.000000
3      0.000000  2.079442  2.079442    0.000000  0.000000
4      0.000000  0.000000  0.000000    1.386294  0.000000
5      0.000000  0.000000  0.000000    2.079442  1.098612


__See full output in text format [here](https://github.com/anishLearnsToCode/bow-representation/blob/master/assets/tfidf.txt).__